In [ ]:
import numpy as np
import pandas as pd


In [ ]:
books = pd.read_csv('/Users/anushkahadkhale/Desktop/Ml_project/Book_recomendation_sys/file/Books.csv')
users = pd.read_csv('/Users/anushkahadkhale/Desktop/Ml_project/Book_recomendation_sys/file/Users.csv')
ratings= pd.read_csv('/Users/anushkahadkhale/Desktop/Ml_project/Book_recomendation_sys/file/Ratings.csv')

In [ ]:
#Popularity Based Recommender system
ratings_with_name = ratings.merge(books,on='ISBN')
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

In [ ]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg-rating'},inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title') 
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg-rating',ascending=False).head(50)
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg-rating']]



In [ ]:
#collaborative recommending syetem
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
user_ratings = x[x].index
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(user_ratings)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)

In [44]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:6]
    for i in similar_items:
        print(pt.index[i[0]])
recommend('1984')


Animal Farm
The Handmaid's Tale
Brave New World
The Vampire Lestat (Vampire Chronicles, Book II)
The Hours : A Novel
